<a href="https://colab.research.google.com/github/malinphy/two_stage_recommender/blob/main/two_stage_recommender_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install --quiet scann==1.2.8

Mounted at /content/drive
     |████████████████████████████████| 10.4 MB 4.6 MB/s 
     |████████████████████████████████| 578.0 MB 10 kB/s 
     |████████████████████████████████| 5.9 MB 41.3 MB/s 
     |████████████████████████████████| 438 kB 54.1 MB/s 
     |████████████████████████████████| 1.7 MB 53.3 MB/s 


# RETRIEVAL STAGE

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Model,layers,Input
from tensorflow.keras.layers import *
import sklearn 
from HelperFunctions import time_conv,time_sorter,time_splitter,last_n_taker,unique_definer,corpus_creator,sequencer_multi,sequencer_unique,input_label_maker
from HelperFunctions import train_neg_maker,release_year
from metrics import mapk
from models import ret_model
import os 
from sklearn.utils import shuffle
import pickle
import scann
from ast import literal_eval

In [3]:
EMBEDDING_DIM = 256
SEQUENCE_LEN = 20
BATCH_SIZE = 512
NUM_EPOCHS = 100
K = 5

saving_path = 'drive/MyDrive/Colab Notebooks/two_stage_rec_2/'

In [4]:
movie_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml_1M/movies.dat'
movies_df = pd.read_csv(movie_url, delimiter = '::',encoding='ISO-8859-1',header = None)
movies_df.columns = ['movie_id','movie_title','genres']

ratings_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml_1M/ratings.dat'
ratings_df = pd.read_csv(ratings_url, delimiter = '::', header = None)
ratings_df.columns = ['user_id','movie_id','ratings','timestamp']

user_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml_1M/users.dat'
users_df = pd.read_csv(user_url, delimiter = '::', header = None)
users_df.columns = ['user_id','gender', 'age', 'occupation', 'zip_code']

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
def pickle_dumper(dictin,dictin_name):
    name_str = dictin_name+'.pkl'
    a_file = open(name_str, "wb")
    pickle.dump(dictin, a_file)
    a_file.close()

In [6]:
df = movies_df.merge(ratings_df, right_on = 'movie_id', left_on = 'movie_id')
df = users_df.merge(df, right_on = 'user_id', left_on = 'user_id')
### sorting each rows as a function of ascending time for each user_id
df['timestamp'] = time_conv(df['timestamp'])
df = time_sorter(df,'user_id', 'timestamp')

df.head(2)

,user_id,timestamp,gender,age,occupation,zip_code,movie_id,movie_title,genres,ratings
0,1,2000-12-31 22:00:19,F,1,10,48067,3186,"Girl, Interrupted (1999)",Drama,4
1,1,2000-12-31 22:00:55,F,1,10,48067,1022,Cinderella (1950),Animation|Children's|Musical,5


In [7]:
### every user has different amount of movie. For every user same amount(sequence_len) of movie has ben selected using last_n_taker function. 
### as an alternative method each movie can be sorted as function of and 0 padding method can be applied
df_diluted = last_n_taker(df, 'user_id', SEQUENCE_LEN)

print('length of the retrieval_df_diluted (every user has same amount of movie) :',len(df_diluted))

### finding unique entries for each features, this part will be used for creating corpus.
unique_users, num_unique_users = unique_definer(df_diluted,'user_id')
unique_movies, num_unique_movies = unique_definer(df_diluted,'movie_id')
unique_occupations, num_unique_occupations = unique_definer(df_diluted, 'occupation')
unique_zips, num_unique_zips = unique_definer(df_diluted, 'zip_code')



### creating corpus for each feature 
### for some features label encoders will be employed
#**************BU KISIMDA KI ENCODER SOZLUKLERI DEGISECEK*******************
user_2enc, enc_2user = corpus_creator(unique_users, start_index = 0)
movie_2enc, enc_2movie = corpus_creator(unique_movies, start_index = 0)
occ_2enc, enc_2occ = corpus_creator(unique_occupations, start_index = 0)
zip_2enc, enc_2zip = corpus_creator(unique_zips, start_index = 0)
mid_2title = { df_diluted['movie_id'][i]:df_diluted['movie_title'][i] for i in range(len(df_diluted))}
title_2mid = { df_diluted['movie_title'][i]:df_diluted['movie_id'][i] for i in range(len(df_diluted))}
#**************BU KISIMDA KI ENCODER SOZLUKLERI DEGISECEK*******************
LE_gender = LabelEncoder()
LE_age = LabelEncoder()
LE_ge = LabelEncoder()


length of the retrieval_df_diluted (every user has same amount of movie) : 120800


In [8]:
df_diluted['movie_id_enc'] = df_diluted['movie_id'].map(enc_2movie) 
df_diluted['occupation_enc'] = df_diluted['occupation'].map(enc_2occ) 
df_diluted['gender_enc'] = LE_gender.fit_transform(df_diluted['gender']) 
df_diluted['age_enc'] = LE_age.fit_transform(df_diluted['age']) 
df_diluted['zip_code_enc'] = df_diluted['zip_code'].map(enc_2zip) 

In [9]:
df_diluted.head(2)

,user_id,timestamp,gender,age,occupation,zip_code,movie_id,movie_title,genres,ratings,movie_id_enc,occupation_enc,gender_enc,age_enc,zip_code_enc
0,1,2000-12-31 22:36:14,F,1,10,48067,3114,Toy Story 2 (1999),Animation|Children's|Comedy,4,0,0,0,0,0
1,1,2000-12-31 22:36:28,F,1,10,48067,2791,Airplane! (1980),Comedy,4,1,0,0,0,0


In [10]:
seq_retrieval_df_enc = sequencer_multi(df_diluted[['user_id','movie_id_enc']],'user_id')
seq_retrieval_df_enc_2 = sequencer_unique(df_diluted,'user_id')
seq_retrieval_df_enc['occupation_enc'] = seq_retrieval_df_enc_2['occupation_enc']
seq_retrieval_df_enc['gender_enc'] = seq_retrieval_df_enc_2['gender_enc']
seq_retrieval_df_enc['age_enc'] = seq_retrieval_df_enc_2['age_enc']
seq_retrieval_df_enc['zip_code_enc'] = seq_retrieval_df_enc_2['zip_code_enc']

del seq_retrieval_df_enc_2

In [11]:
#### sequenced movies will split into test and train set using leave one out methodology,
#### last watched movie will be designated as label item or label movie
last_movie_enc, input_movie_enc = input_label_maker(seq_retrieval_df_enc,'movie_id_enc',SEQUENCE_LEN, splitter = True)
seq_retrieval_df_enc['last_movie_enc'] = last_movie_enc
seq_retrieval_df_enc['input_movie_enc'] = input_movie_enc

#### for testing purpose dataset will be arraged as train and test sets 
train_df, test_df = train_test_split(seq_retrieval_df_enc, train_size = 0.96, test_size = 0.04, random_state = 42)

#### data types will be converted appropriate format for neural network
input_movie_enc_train = [np.array(i)  for i in (train_df['input_movie_enc'])]
last_movie_enc_train = [np.array(i)  for i in (train_df['last_movie_enc'])]
occupation_enc_train = [np.array(i)  for i in (train_df['occupation_enc'])]
gender_enc_train = [np.array(i)  for i in (train_df['gender_enc'])]
age_enc_train = [np.array(i)  for i in (train_df['age_enc'])]
zip_code_enc_train = [np.array(i)  for i in (train_df['zip_code_enc'])]

input_movie_enc_test = [np.array(i)  for i in (test_df['input_movie_enc'])]
last_movie_enc_test = [np.array(i)  for i in (test_df['last_movie_enc'])]
occupation_enc_test = [np.array(i)  for i in (test_df['occupation_enc'])]
gender_enc_test = [np.array(i)  for i in (test_df['gender_enc'])]
age_enc_test = [np.array(i)  for i in (test_df['age_enc'])]
zip_code_enc_test = [np.array(i)  for i in (test_df['zip_code_enc'])]

input_movie_enc = [list(i)  for i in (seq_retrieval_df_enc['input_movie_enc'])]
last_movie_enc = [np.array(i)  for i in (seq_retrieval_df_enc['last_movie_enc'])]
occupation_enc = [list(i)  for i in (seq_retrieval_df_enc['occupation_enc'])]
gender_enc = [list(i)  for i in (seq_retrieval_df_enc['gender_enc'])]
age_enc = [list(i)  for i in (seq_retrieval_df_enc['age_enc'])]
zip_code_enc = [list(i)  for i in (seq_retrieval_df_enc['zip_code_enc'])]


In [12]:
retrieval_model = ret_model(SEQUENCE_LEN,EMBEDDING_DIM,BATCH_SIZE,NUM_EPOCHS,num_unique_movies)
retrieval_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = 'Adam',
    metrics = ['accuracy'],
    )

retrieval_model.fit([
        tf.constant(input_movie_enc_train),
        tf.constant(occupation_enc_train),
        tf.constant(gender_enc_train),
        tf.constant(age_enc_train),
        tf.constant(zip_code_enc_train)
        ],
        tf.constant(last_movie_enc_train),
     epochs = NUM_EPOCHS,
    #  batch_size = BATCH_SIZE,
    verbose = 1
     )

### One can save either model or model weights. Since, model weight takes less space, I prefer to save model weights rather that whole model
ret_model_name = 'retrieval_model_weights.h5'

# retrieval_model.save_weights(saving_path +str(NUM_EPOCHS)+'epochs_'+ret_model_name)
# retrieval_model.load_weights(saving_path +str(NUM_EPOCHS)+'epochs_'+ret_model_name)
# retrieval_model.load_weights('drive/MyDrive/Colab Notebooks/two_stage_rec/retrieval_weights.h5')

Epoch 1/100
182/182 [==============================] - 20s 107ms/step - loss: 8.4152 - accuracy: 0.0076
Epoch 2/100
182/182 [==============================] - 15s 84ms/step - loss: 7.9571 - accuracy: 0.0109
Epoch 3/100
182/182 [==============================] - 15s 84ms/step - loss: 7.8405 - accuracy: 0.0102
Epoch 4/100
182/182 [==============================] - 18s 101ms/step - loss: 7.7438 - accuracy: 0.0126
Epoch 5/100
182/182 [==============================] - 15s 83ms/step - loss: 7.6402 - accuracy: 0.0143
Epoch 6/100
182/182 [==============================] - 16s 86ms/step - loss: 7.5069 - accuracy: 0.0162
Epoch 7/100
182/182 [==============================] - 16s 88ms/step - loss: 7.3720 - accuracy: 0.0160
Epoch 8/100
182/182 [==============================] - 16s 85ms/step - loss: 7.3189 - accuracy: 0.0197
Epoch 9/100
182/182 [==============================] - 15s 83ms/step - loss: 7.2055 - accuracy: 0.0243
Epoch 10/100
182/182 [==============================] - 15s 82ms/step -

In [13]:
## As known, recommender systems based on the hidden features of the user and item embeddings. Therefore, system needs generate
##  user embeddings and item embeddings. User embeddings were generated as weights of the softmax layer.
## item embeddings were generated as output of the dense layer (not weights) before the softmax layer.
## To generate the item embeddings, I decided to cancel softmax layer and constract another model with trained weigths of the previous model and named it as user_emb_model
item_embeddings = ((retrieval_model.get_layer("softmax_layer").weights)[0])
user_emb_model = Model(inputs = retrieval_model.inputs , outputs = retrieval_model.get_layer('d3_layer').output )
user_embeddings = user_emb_model([
                                            tf.constant(input_movie_enc),
                                            tf.constant(occupation_enc),
                                            tf.constant(gender_enc),
                                            tf.constant(age_enc),
                                            tf.constant(zip_code_enc)
                                            ])

## at this point, vector database is created using google scaNN and item_embeddings were inserted in it.
searcher = scann.scann_ops_pybind.builder(np.transpose(item_embeddings), K, "dot_product").tree(
    num_leaves=20, num_leaves_to_search=10, training_sample_size=2000
    ).score_ah(2, anisotropic_quantization_threshold=0.2
    ).reorder(100).build()

## create serialize target dir
# saving_path = 'drive/MyDrive/Colab Notebooks/two_stage_rec'
saving_path_searcher = 'drive/MyDrive/Colab Notebooks/two_stage_rec_3/searcher'
os.makedirs(saving_path_searcher, exist_ok=True)
# ## serialize the searcher
searcher.serialize(saving_path_searcher)

searcher = scann.scann_ops_pybind.load_searcher(saving_path_searcher)

In [14]:
top_decoded_items = []
topN_enc_item_id = []
user_id = 1
user_id_col = []
for i in user_embeddings:
    index, distance = searcher.search(np.array(i).ravel())
    topN_enc_item_id.append(index)
    user_id_col.append(np.full(len(index),user_id))
    user_id += 1

In [15]:
### bu kisimda ranking df icin dataset hazirliniyor
### BU KISMI TEKRAR KONTROL ET. CUNKU BIR YERDE ITEM EMBEDDINGS VE MOVIE EMBEDDING DEGISTIRILMIS. 
retrieval_final_df = pd.DataFrame({'user_id':np.concatenate(user_id_col) , 'movie_id_enc':np.concatenate(topN_enc_item_id)})
retrieval_final_df['user_id'] = retrieval_final_df['user_id']
retrieval_final_df['movie_id'] = retrieval_final_df['movie_id_enc'].map(movie_2enc)
ranking_df = retrieval_final_df.copy()
ranking_df = ranking_df.drop_duplicates()
ranking_df = ranking_df.rename(columns = {'movie_id':'item_id'})
ranking_df = ranking_df.drop_duplicates()
ranking_df = ranking_df.merge(movies_df, left_on='item_id', right_on= 'movie_id')
ranking_df = ranking_df.drop_duplicates().reset_index(drop= True)

In [16]:
top_decoded_items = []
topN_enc_item_id = []
user_id = 1
user_id_col = []

for i in user_embeddings:
    index, distance = searcher.search(np.array(i).ravel())
    topN_enc_item_id.append(index)
    user_id_col.append(np.full(len(index),user_id))
    user_id += 1
print('MAP',mapk(np.array(last_movie_enc_test).reshape(-1,1), np.concatenate(topN_enc_item_id).reshape(-1,1), k = 20))

def _compute_precision_recall(targets, predictions, k):

    pred = predictions[:k]
    num_hit = len(set(pred).intersection(set(targets)))
    precision = float(num_hit) / len(pred)
    recall = float(num_hit) / len(targets)
    return precision, recall

N = [1,3,5,10,20]
for t in N:
    precisions = []
    recalls = []
    for i, _k in enumerate(topN_enc_item_id):
        precision, recall = _compute_precision_recall([int(last_movie_enc[i])], _k,t)
    # print(precision)
        precisions.append(precision)
        recalls.append(recall)

    print('precision @',t, np.mean(precisions))
    print('recalls @',t, np.mean(recalls))
    print(' ')

MAP 0.0
precision @ 1 0.3622516556291391
recalls @ 1 0.3622516556291391
 
precision @ 3 0.14199779249448122
recalls @ 3 0.42599337748344374
 
precision @ 5 0.09056291390728477
recalls @ 5 0.45281456953642385
 
precision @ 10 0.09056291390728477
recalls @ 10 0.45281456953642385
 
precision @ 20 0.09056291390728477
recalls @ 20 0.45281456953642385
 


In [17]:
def predictor(user_id):
    user_embeddings[user_id]
    index, distance = searcher.search(np.array(user_embeddings[user_id]).ravel())

    return index

In [18]:
# ### we another dataset to train the ranking system with selected top_N recommended items
# df_diluted_index = df_diluted.set_index('movie_id')
# # df_diluted_index.head(3)
# diluted_index = np.unique(np.concatenate(topN_enc_item_id))
# selected_index = [movie_2enc[i] for i in diluted_index]
# ranking_df = df_diluted_index.loc[selected_index].reset_index()
# ranking_df = ranking_df.sort_values('user_id')
# # ranking_df.to_csv('drive/MyDrive/Colab Notebooks/two_stage_rec_3/ranking_df.csv')
# # ranking_df

# a_file = open("movie_2enc.pkl", "wb")
# pickle.dump(movie_2enc, a_file)
# a_file.close()


# # a_file = open("movie_2enc.pkl", "rb")
# # output = pickle.load(a_file)
# # print(output)

In [19]:
# !pip install --quiet scann==1.2.8
# import scann
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Model,layers,Input
from tensorflow.keras.layers import *
from HelperFunctions import time_conv,time_sorter,time_splitter,last_n_taker,unique_definer,corpus_creator,sequencer_multi,sequencer_unique,input_label_maker
from HelperFunctions import train_neg_maker,release_year
from HelperFunctions_ranking import genre_splitter, date_extractor
from metrics import mapk
from models import ret_model,ranking_model
import os 
import sklearn
from sklearn.utils import shuffle
import pickle


import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import preprocessing

In [20]:
EMBEDDING_DIM = 256
SEQUENCE_LEN = 20
BATCH_SIZE = 512
NUM_EPOCHS = 100
K = 40
# unique_items_rank_len = 3366
NUM_EPOCHS_RANKING = 5

def pickle_dumper(dictin,dictin_name):
    name_str = dictin_name+'.pkl'
    a_file = open(name_str, "wb")
    pickle.dump(dictin, a_file)
    a_file.close()

In [21]:
ranking_df = pd.read_csv('drive/MyDrive/Colab Notebooks/two_stage_rec_3/ranking_df.csv')

ranking_df = ranking_df.drop(columns = ['Unnamed: 0','timestamp','gender','age','occupation',
                                        'zip_code','movie_id_enc','occupation_enc',
                                        'gender_enc','age_enc','zip_code_enc'])


In [22]:
unique_movies_rank, num_unique_movies_rank = unique_definer(ranking_df,'movie_id')
movie_2enc_rank, enc_2movie_rank = corpus_creator(unique_movies_rank, start_index = 0)

movie_id_2_title =  {ranking_df['movie_id'][i]:ranking_df['movie_title'][i] for i in range(len(ranking_df))  }
movie_title_2_id =  {ranking_df['movie_title'][i]:ranking_df['movie_id'][i] for i in range(len(ranking_df))  }
movie_id_2_genres = {ranking_df['movie_id'][i]:ranking_df['genres'][i] for i in range(len(ranking_df))  }
genres_2_movie_id = {ranking_df['genres'][i]:ranking_df['movie_id'][i] for i in range(len(ranking_df))  }

In [23]:
ranking_df['movie_id_enc'] = ranking_df['movie_id'].map(enc_2movie_rank)

In [24]:
sequenced_df = sequencer_multi(ranking_df,'user_id')
sequenced_df.head(3)
sequenced_df = sequenced_df.drop(columns = ['movie_title','genres'])
sequenced_df.head(3)

,user_id,movie_id,ratings,movie_id_enc
0,1,"[3114, 2294, 2687, 745, 595, 588, 1907, 2355, ...","[4, 4, 3, 3, 5, 4, 4, 5, 5, 4, 3, 5, 5, 4, 4, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,2,"[1801, 1544, 442, 3107, 1597, 95, 2126, 368, 2...","[3, 4, 3, 2, 3, 2, 3, 4, 3, 3, 3, 4, 3, 3]","[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2..."
2,3,"[104, 3868, 648, 1259, 3552, 2735, 1136, 1641,...","[4, 3, 3, 5, 5, 4, 5, 2, 4, 3, 4, 2, 5, 3, 5, ...","[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4..."


In [25]:
users, negatives = train_neg_maker(4, sequenced_df,'movie_id_enc',movie_2enc_rank)
negatives = [np.concatenate(i) for i in negatives]
negatives = np.concatenate(negatives)
negative_df_rank = pd.DataFrame({'user_id':np.add(users,1),'movie_id_enc':negatives})
negative_df_rank['ratings'] = np.zeros(len(negative_df_rank), dtype = 'int32')
negative_df_rank = negative_df_rank.reset_index(drop=True)

negative_df_rank['movie_id'] = negative_df_rank['movie_id_enc'].map(movie_2enc_rank)
negative_df_rank['movie_title'] = negative_df_rank['movie_id'].map(movie_id_2_title)
negative_df_rank['genres'] = negative_df_rank['movie_id'].map(movie_id_2_genres)

ranking_df['ratings'] = np.ones(len(ranking_df),dtype = 'int32')

ranking_train_df = pd.concat([negative_df_rank,ranking_df])


In [26]:
genres,total_genre = genre_splitter(ranking_train_df['genres'])
unique_genres = np.unique(np.concatenate(genres),return_counts =True)[0]

mlb = MultiLabelBinarizer()
mlb_fits = mlb.fit_transform(genres)
mlb.classes_

release_date = date_extractor(ranking_train_df['movie_title'])
np.unique(release_date,return_counts = True)

genre_enc = [list(i) for i in mlb_fits]
ranking_train_df['genre_enc'] = genre_enc
ranking_train_df['genre_count'] = total_genre
ranking_train_df['release_date'] = release_date
ranking_train_df['release_date_norm'] = preprocessing.normalize([ranking_train_df['release_date']])[0]

In [27]:
genre_size = 18
x= tf.keras.preprocessing.sequence.pad_sequences(ranking_train_df['genre_enc'], padding="post" ,maxlen = 18)
x = [list(i) for i in (x)]
ranking_train_df['genre_enc'] = x

In [28]:
ranking_train_df =  ranking_train_df.reset_index(drop =True)
ranking_train_df = ranking_train_df.dropna().reset_index(drop = True)
unique_items_rank = ranking_train_df['movie_id_enc'].unique()

input_user_id_rank = [np.array(i)  for i in (ranking_train_df['user_id'])]
input_movie_rank = [np.array(i)  for i in (ranking_train_df['movie_id_enc'])]
input_release_year_rank = [np.array(i, dtype = 'int32')  for i in (ranking_train_df['release_date'])]
input_genre_rank = [list(i) for i in (ranking_train_df['genre_enc'])]
input_genre_count_rank = [np.array(i)  for i in (ranking_train_df['genre_count'])]
input_release_date_norm_rank = [np.array(i)  for i in (ranking_train_df['release_date_norm'])]


In [29]:
unique_items_rank_len =len(unique_items_rank)

In [30]:
ranking_model = ranking_model(unique_items_rank_len,EMBEDDING_DIM,genre_size = 18)
ranking_model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = 'Adam',
    metrics = ['accuracy'],
)
ranking_model.fit(
    [tf.constant(input_user_id_rank),
     tf.constant(input_movie_rank),
     tf.constant(input_release_year_rank),
     tf.constant(input_genre_rank),
     tf.constant(input_genre_count_rank),
     tf.constant(input_release_date_norm_rank)
     ],
     tf.constant(ranking_train_df['ratings']),
     epochs = 10,
     batch_size = 256
)

Epoch 1/10
1889/1889 [==============================] - 94s 49ms/step - loss: 1.5553 - accuracy: 0.7758
Epoch 2/10
1889/1889 [==============================] - 86s 46ms/step - loss: 0.4791 - accuracy: 0.7994
Epoch 3/10
1889/1889 [==============================] - 90s 48ms/step - loss: 0.4426 - accuracy: 0.8027
Epoch 4/10
1889/1889 [==============================] - 85s 45ms/step - loss: 0.4398 - accuracy: 0.8036
Epoch 5/10
1889/1889 [==============================] - 94s 50ms/step - loss: 0.4438 - accuracy: 0.8026
Epoch 6/10
1889/1889 [==============================] - 86s 45ms/step - loss: 0.4390 - accuracy: 0.8018
Epoch 7/10
1889/1889 [==============================] - 90s 48ms/step - loss: 0.4371 - accuracy: 0.8030
Epoch 8/10
1889/1889 [==============================] - 90s 48ms/step - loss: 0.4360 - accuracy: 0.8039
Epoch 9/10
1889/1889 [==============================] - 88s 47ms/step - loss: 0.4346 - accuracy: 0.8051
Epoch 10/10
1889/1889 [==============================] - 87s 46m

In [31]:
selected_user_id = 6
selected_df = ranking_train_df[ranking_train_df['user_id'] == selected_user_id]
selected_index = selected_df[selected_df['ratings'] == 1].index
selected_index

selected_preds = ranking_model.predict([
    tf.constant(np.array(input_user_id_rank)[selected_index]),
    tf.constant(np.array(input_movie_rank)[selected_index]),
    tf.constant(np.array(input_release_year_rank)[selected_index]),
    tf.constant(np.array(input_genre_rank)[selected_index]),
    tf.constant(np.array(input_genre_count_rank)[selected_index]),
    tf.constant(np.array(input_release_date_norm_rank)[selected_index])
     ])

np.argsort(-np.transpose(np.array(selected_preds)))
print(selected_df.reset_index(drop=True).iloc[[3,6,12]]['movie_id'])

1/1 [==============================] - 0s 152ms/step
3     1042
6      261
12     509
Name: movie_id, dtype: int64
